In [3]:
import pandas as pd
import numpy as np
import os, sys
import math
import copy

## PROCESS FIRST SAMPLE

In [11]:
input = '''2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533'''

In [20]:
opposite_direction = {'r':'l', 'l':'r', 'u':'d', 'd':'u'}
next_movement = {'r':[0,1], 'l':[0,-1], 'u':[-1,0], 'd':[1,0]}

In [13]:
input = input.split('\n')
map = []
for line in input:
    print(line)
    map.append([int(x) for x in line])
map
np_map = np.array(map)
np_map

2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533


array([[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3],
       [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3],
       [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4],
       [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2],
       [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6],
       [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4],
       [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6],
       [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3],
       [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7],
       [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3],
       [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3],
       [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5],
       [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]])

In [45]:
position = (0,0)
active_paths = [(position,'d',0,0)] # POSITION, DIRECTION, STREAK, HEAT
optimal_visits = {}                 # DICT TO STORE OPTIMAL HEATS PER [POSITION] [(DIRECTION, STREAK)]

while len(active_paths) > 0:
    current_path = active_paths.pop(0)
    current_position, current_direction, current_streak, current_heat = current_path

    next_heat = np_map[current_position[0], current_position[1]] + current_heat
    optimal_visits_key = (current_direction, current_streak)
    
    if current_position in optimal_visits.keys():
        if optimal_visits_key in optimal_visits[current_position].keys():
            if optimal_visits[current_position][optimal_visits_key] <= next_heat: # WE ONLY CONTINUE PATHS THAT IMPROVE ON EXISTING KNOWLEDGE
                continue
    else:
        optimal_visits[current_position] = {}
    
    optimal_visits[current_position][optimal_visits_key] = next_heat      # EITHER NEW KEY OR IMPROVING KNOWN HEAT

    next_direction_list = list(opposite_direction.keys())
    next_direction_list.remove(opposite_direction[current_direction])
    if current_streak == 3:                             # SAME DIRECTION CANNOT GO BEYOND STREAK 3
        next_direction_list.remove(current_direction)
    
    for next_direction in next_direction_list:
        next_position = tuple(np.add(current_position, next_movement[next_direction]))
        if (next_position[0] < 0) or (next_position[0] >= len(map) or next_position[1] < 0 or next_position[1] >= len(map[0])): # DO NOT PURSUE OUT OF BOUNDS POSITION
            continue
        if next_direction == current_direction:
            next_streak = current_streak + 1
        else:
            next_streak = 1

        active_paths.append((next_position,next_direction,next_streak,next_heat))
    

In [46]:
optimal_visits[(0,0)], optimal_visits[(12,12)]

({('d', 0): 2,
  ('u', 1): 13,
  ('l', 1): 13,
  ('l', 2): 15,
  ('u', 2): 17,
  ('l', 3): 23,
  ('u', 3): 24},
 {('d', 3): 106,
  ('d', 2): 108,
  ('d', 1): 106,
  ('r', 1): 104,
  ('r', 2): 116,
  ('r', 3): 117})

In [44]:
optimal_map = []
for row in range(np_map.shape[0]):
    optimal_row = []
    for col in range(np_map.shape[1]):
        optimal = min(optimal_visits[(row,col)].values())
        optimal_row.append(str(optimal))
    optimal_map.append(optimal_row)
    print(optimal_row)


['2', '6', '7', '10', '16', '19', '25', '29', '31', '34', '40', '42', '48']
['5', '7', '8', '13', '17', '22', '27', '32', '34', '39', '45', '44', '47']
['8', '9', '13', '18', '19', '23', '28', '34', '39', '43', '45', '49', '51']
['11', '13', '17', '23', '24', '31', '33', '41', '43', '48', '49', '54', '53']
['19', '18', '21', '27', '30', '35', '40', '48', '50', '55', '54', '57', '59']
['20', '23', '24', '32', '36', '44', '48', '55', '60', '63', '58', '62', '65']
['25', '28', '29', '36', '44', '51', '56', '64', '68', '70', '68', '68', '72']
['30', '34', '33', '40', '48', '55', '63', '72', '78', '81', '77', '73', '75']
['35', '40', '39', '43', '52', '58', '66', '77', '85', '88', '87', '82', '82']
['43', '46', '47', '47', '53', '60', '69', '83', '91', '96', '91', '91', '86']
['46', '48', '50', '51', '57', '65', '71', '81', '88', '94', '96', '95', '89']
['51', '53', '54', '57', '62', '66', '74', '82', '93', '101', '105', '98', '100']
['55', '56', '56', '58', '64', '71', '77', '83', '88', '9

In [48]:
solution = '''2>>34^>>>1323
32v>>>35v5623
32552456v>>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>'''

In [49]:
solution = solution.split('\n')
solution_map = []
for line in solution:
    print(line)
    solution_map.append([x for x in line])
solution_map
solution_map = np.array(solution_map)
solution_map

2>>34^>>>1323
32v>>>35v5623
32552456v>>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>


array([['2', '>', '>', '3', '4', '^', '>', '>', '>', '1', '3', '2', '3'],
       ['3', '2', 'v', '>', '>', '>', '3', '5', 'v', '5', '6', '2', '3'],
       ['3', '2', '5', '5', '2', '4', '5', '6', 'v', '>', '>', '5', '4'],
       ['3', '4', '4', '6', '5', '8', '5', '8', '4', '5', 'v', '5', '2'],
       ['4', '5', '4', '6', '6', '5', '7', '8', '6', '7', 'v', '>', '6'],
       ['1', '4', '3', '8', '5', '9', '8', '7', '9', '8', '4', 'v', '4'],
       ['4', '4', '5', '7', '8', '7', '6', '9', '8', '7', '7', 'v', '6'],
       ['3', '6', '3', '7', '8', '7', '7', '9', '7', '9', '6', 'v', '>'],
       ['4', '6', '5', '4', '9', '6', '7', '9', '8', '6', '8', '8', 'v'],
       ['4', '5', '6', '4', '6', '7', '9', '9', '8', '6', '4', '5', 'v'],
       ['1', '2', '2', '4', '6', '8', '6', '8', '6', '5', '5', '<', 'v'],
       ['2', '5', '4', '6', '5', '4', '8', '8', '8', '7', '7', 'v', '5'],
       ['4', '3', '2', '2', '6', '7', '4', '6', '5', '5', '5', 'v', '>']],
      dtype='<U1')

## PROCESS PART 1

In [57]:
# READ FILE
inputfilepath = './input.txt'

input = []
with open(inputfilepath,'r') as f:
    for line in f:
        input.append(line.rstrip())

In [58]:
map = []
for line in input:
    map.append([int(x) for x in line])
np_map = np.array(map)
print(np_map.shape)
np_map

(141, 141)


array([[3, 1, 4, ..., 5, 5, 5],
       [5, 4, 1, ..., 5, 3, 5],
       [3, 3, 1, ..., 5, 4, 2],
       ...,
       [5, 5, 2, ..., 3, 1, 3],
       [2, 2, 1, ..., 1, 3, 2],
       [1, 4, 4, ..., 4, 1, 5]])

In [60]:
position = (0,0)
active_paths = [(position,'d',0,0,[position])] # POSITION, DIRECTION, STREAK, HEAT, PATH_SO_FAR
optimal_visits = {}                 # DICT TO STORE OPTIMAL HEATS PER [POSITION] [(DIRECTION, STREAK)]

while len(active_paths) > 0:
    min_heat = active_paths[0][3]
    min_id = 0
    for i_path, path in enumerate(active_paths):
        if path[3] < min_heat:
            min_heat = path[3]
            min_id = i_path
    current_path = active_paths.pop(min_id)
    current_position, current_direction, current_streak, current_heat, current_pathsofar = current_path

    next_heat = np_map[current_position[0], current_position[1]] + current_heat
    optimal_visits_key = (current_direction, current_streak)
    
    if current_position in optimal_visits.keys():
        if optimal_visits_key in optimal_visits[current_position].keys():
            if optimal_visits[current_position][optimal_visits_key] <= next_heat: # WE ONLY CONTINUE PATHS THAT IMPROVE ON EXISTING KNOWLEDGE
                continue
    else:
        optimal_visits[current_position] = {}
    
    optimal_visits[current_position][optimal_visits_key] = next_heat      # EITHER NEW KEY OR IMPROVING KNOWN HEAT

    next_direction_list = list(opposite_direction.keys())
    next_direction_list.remove(opposite_direction[current_direction])
    if current_streak == 3:                             # SAME DIRECTION CANNOT GO BEYOND STREAK 3
        next_direction_list.remove(current_direction)
    
    for next_direction in next_direction_list:
        next_position = tuple(np.add(current_position, next_movement[next_direction]))
        if (next_position[0] < 0) or (next_position[0] >= len(map) or next_position[1] < 0 or next_position[1] >= len(map[0])): # DO NOT PURSUE OUT OF BOUNDS POSITION
            continue
        if next_position in current_pathsofar:  # DO NOT BACKTRACK
            continue

        if next_direction == current_direction:
            next_streak = current_streak + 1
        else:
            next_streak = 1
        
        next_pathsofar = current_pathsofar.copy()
        next_pathsofar.append(next_position)

        active_paths.append((next_position,next_direction,next_streak,next_heat,next_pathsofar))

In [65]:
min(optimal_visits[(140,140)].values()) - min(optimal_visits[(0,0)].values())

1256

## PROCESS SECOND SAMPLE

In [66]:
input = '''2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533'''

In [67]:
input = input.split('\n')
map = []
for line in input:
    print(line)
    map.append([int(x) for x in line])
map
np_map = np.array(map)
np_map

2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533


array([[2, 4, 1, 3, 4, 3, 2, 3, 1, 1, 3, 2, 3],
       [3, 2, 1, 5, 4, 5, 3, 5, 3, 5, 6, 2, 3],
       [3, 2, 5, 5, 2, 4, 5, 6, 5, 4, 2, 5, 4],
       [3, 4, 4, 6, 5, 8, 5, 8, 4, 5, 4, 5, 2],
       [4, 5, 4, 6, 6, 5, 7, 8, 6, 7, 5, 3, 6],
       [1, 4, 3, 8, 5, 9, 8, 7, 9, 8, 4, 5, 4],
       [4, 4, 5, 7, 8, 7, 6, 9, 8, 7, 7, 6, 6],
       [3, 6, 3, 7, 8, 7, 7, 9, 7, 9, 6, 5, 3],
       [4, 6, 5, 4, 9, 6, 7, 9, 8, 6, 8, 8, 7],
       [4, 5, 6, 4, 6, 7, 9, 9, 8, 6, 4, 5, 3],
       [1, 2, 2, 4, 6, 8, 6, 8, 6, 5, 5, 6, 3],
       [2, 5, 4, 6, 5, 4, 8, 8, 8, 7, 7, 3, 5],
       [4, 3, 2, 2, 6, 7, 4, 6, 5, 5, 5, 3, 3]])

In [68]:
position = (0,0)
active_paths = [(position,'r',0,0,[position])] # POSITION, DIRECTION, STREAK, HEAT, PATH_SO_FAR
optimal_visits = {}                 # DICT TO STORE OPTIMAL HEATS PER [POSITION] [(DIRECTION, STREAK)]

while len(active_paths) > 0:
    min_heat = active_paths[0][3]
    min_id = 0
    for i_path, path in enumerate(active_paths):
        if path[3] < min_heat:
            min_heat = path[3]
            min_id = i_path
    current_path = active_paths.pop(min_id)
    current_position, current_direction, current_streak, current_heat, current_pathsofar = current_path

    next_heat = np_map[current_position[0], current_position[1]] + current_heat
    optimal_visits_key = (current_direction, current_streak)
    
    if current_position in optimal_visits.keys():
        if optimal_visits_key in optimal_visits[current_position].keys():
            if optimal_visits[current_position][optimal_visits_key] <= next_heat: # WE ONLY CONTINUE PATHS THAT IMPROVE ON EXISTING KNOWLEDGE
                continue
    else:
        optimal_visits[current_position] = {}
    
    optimal_visits[current_position][optimal_visits_key] = next_heat      # EITHER NEW KEY OR IMPROVING KNOWN HEAT

    next_direction_list = list(opposite_direction.keys())
    next_direction_list.remove(opposite_direction[current_direction])
    if current_streak == 10:                             # SAME DIRECTION CANNOT GO BEYOND STREAK 3
        next_direction_list.remove(current_direction)
    if current_streak < 4:
        next_direction_list = [current_direction]
    
    for next_direction in next_direction_list:
        next_position = tuple(np.add(current_position, next_movement[next_direction]))
        if (next_position[0] < 0) or (next_position[0] >= len(map) or next_position[1] < 0 or next_position[1] >= len(map[0])): # DO NOT PURSUE OUT OF BOUNDS POSITION
            continue
        if next_position in current_pathsofar:  # DO NOT BACKTRACK
            continue

        if next_direction == current_direction:
            next_streak = current_streak + 1
        else:
            next_streak = 1
        
        next_pathsofar = current_pathsofar.copy()
        next_pathsofar.append(next_position)

        active_paths.append((next_position,next_direction,next_streak,next_heat,next_pathsofar))

In [71]:
optimal_visits[(12,12)], min(optimal_visits[(0,0)].values())

({('d', 8): 96,
  ('d', 7): 101,
  ('d', 6): 108,
  ('d', 5): 109,
  ('d', 3): 110,
  ('r', 1): 112,
  ('d', 2): 114,
  ('d', 4): 116,
  ('r', 2): 117,
  ('r', 3): 128,
  ('r', 4): 132,
  ('d', 1): 136,
  ('r', 10): 136,
  ('r', 8): 147,
  ('r', 7): 149,
  ('r', 6): 153,
  ('d', 10): 155,
  ('r', 9): 159,
  ('r', 5): 160,
  ('d', 9): 165},
 2)

## PROCESS PART 2

In [72]:
# READ FILE
inputfilepath = './input.txt'

input = []
with open(inputfilepath,'r') as f:
    for line in f:
        input.append(line.rstrip())

In [73]:
map = []
for line in input:
    map.append([int(x) for x in line])
np_map = np.array(map)
print(np_map.shape)
np_map

(141, 141)


array([[3, 1, 4, ..., 5, 5, 5],
       [5, 4, 1, ..., 5, 3, 5],
       [3, 3, 1, ..., 5, 4, 2],
       ...,
       [5, 5, 2, ..., 3, 1, 3],
       [2, 2, 1, ..., 1, 3, 2],
       [1, 4, 4, ..., 4, 1, 5]])

In [75]:
position = (0,0)
active_paths = [(position,'r',0,0,[position])] # POSITION, DIRECTION, STREAK, HEAT, PATH_SO_FAR
optimal_visits = {}                 # DICT TO STORE OPTIMAL HEATS PER [POSITION] [(DIRECTION, STREAK)]

while len(active_paths) > 0:
    min_heat = active_paths[0][3]
    min_id = 0
    for i_path, path in enumerate(active_paths):
        if path[3] < min_heat:
            min_heat = path[3]
            min_id = i_path
    current_path = active_paths.pop(min_id)
    current_position, current_direction, current_streak, current_heat, current_pathsofar = current_path

    next_heat = np_map[current_position[0], current_position[1]] + current_heat
    optimal_visits_key = (current_direction, current_streak)
    
    if current_position in optimal_visits.keys():
        if optimal_visits_key in optimal_visits[current_position].keys():
            if optimal_visits[current_position][optimal_visits_key] <= next_heat: # WE ONLY CONTINUE PATHS THAT IMPROVE ON EXISTING KNOWLEDGE
                continue
    else:
        optimal_visits[current_position] = {}
    
    optimal_visits[current_position][optimal_visits_key] = next_heat      # EITHER NEW KEY OR IMPROVING KNOWN HEAT

    next_direction_list = list(opposite_direction.keys())
    next_direction_list.remove(opposite_direction[current_direction])
    if current_streak == 10:                             # SAME DIRECTION CANNOT GO BEYOND STREAK 3
        next_direction_list.remove(current_direction)
    if current_streak < 4:
        next_direction_list = [current_direction]
    if current_position[0] == np_map.shape[0] and current_position[1] == np_map.shape[1]:   # WE DO NOT MOVE AWAY FROM FINAL CASE
        next_direction_list = []
    
    for next_direction in next_direction_list:
        next_position = tuple(np.add(current_position, next_movement[next_direction]))
        if (next_position[0] < 0) or (next_position[0] >= len(map) or next_position[1] < 0 or next_position[1] >= len(map[0])): # DO NOT PURSUE OUT OF BOUNDS POSITION
            continue
        if next_position in current_pathsofar:  # DO NOT BACKTRACK
            continue

        if next_direction == current_direction:
            next_streak = current_streak + 1
        else:
            next_streak = 1
        
        next_pathsofar = current_pathsofar.copy()
        next_pathsofar.append(next_position)

        active_paths.append((next_position,next_direction,next_streak,next_heat,next_pathsofar))

KeyboardInterrupt: 

In [78]:
import heapq
def minimal_heat(start, end, least, most):
    queue = [(0, *start, 0,0)]
    seen = set()
    while queue:
        heat,x,y,px,py = heapq.heappop(queue)
        if (x,y) == end: return heat
        if (x,y, px,py) in seen: continue
        seen.add((x,y, px,py))
        # calculate turns only
        for dx,dy in {(1,0),(0,1),(-1,0),(0,-1)}-{(px,py),(-px,-py)}:
            a,b,h = x,y,heat
            # enter 4-10 moves in the chosen direction
            for i in range(1,most+1):
                a,b=a+dx,b+dy
                if (a,b) in board:
                    h += board[a,b]
                    if i>=least:
                        heapq.heappush(queue, (h, a,b, dx,dy))

board = {(i,j): int(c) for i,r in enumerate(open('input.txt')) for j,c in enumerate(r.strip())}
print(minimal_heat((0,0),max(board), 1, 3))
print(minimal_heat((0,0),max(board), 4, 10))

1256
1382
